In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

In [ ]:
print("Sample training label:", train_labels[0])
print("Sample training image shape:", train_images[0].shape)

In [ ]:
import numpy as np

labels = np.unique(train_labels)
print("labels are:", labels)

In [ ]:
import matplotlib.pyplot as plt

def plot_sample_images(images, labels, num_samples=5):
    plt.figure(figsize=(10, 2))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.title(f"Label: {labels[i]}")
        plt.axis('off')
    plt.show()
    
plot_sample_images(train_images, train_labels)

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.countplot(x=train_labels)
plt.title("Distribution of Labels in Training Set")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

In [ ]:
def plot_images_by_class(images, labels, num_classes=10, num_samples=5):
    plt.figure(figsize=(15, num_classes * 3))
    for label in range(num_classes):
        class_images = images[labels == label][0]
        for i in range(num_samples):
            plt.subplot(num_classes, num_samples, label * num_samples + i + 1)
            plt.imshow(class_images[i], cmap='gray')
            plt.title(f"Label: {label}")
    plt.show()
    
plot_images_by_class(train_images, train_labels)

In [ ]:
from tensorflow.keras.utils import to_categorical

train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

train_labels_categorical = to_categorical(train_labels)
test_labels_categorical = to_categorical(test_labels)

Model 1: Long Short-Term Memory

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout

model_lstm = Sequential([
    Input(shape=(28, 28)),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

callbacks = [ModelCheckpoint('digit_recognition_model.keras', verbose=1, save_best_only=True)]
history = model_lstm.fit(train_images, train_labels_categorical, epochs=15, batch_size=128, validation_data=(test_images, test_labels_categorical), verbose=1, callbacks=callbacks)

In [ ]:
test_loss, test_accuracy = model_lstm.evaluate(test_images, test_labels_categorical)
print('Test accuracy:', test_accuracy)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
sample_image_lstm = test_images[24]
sample_image_lstm = np.expand_dims(sample_image_lstm, axis=0)
prediction_lstm = model_lstm.predict(sample_image_lstm)
predicted_class_lstm = np.argmax(prediction_lstm[0])

plt.imshow(sample_image_lstm[0], cmap='gray')
plt.title(f"Predicted Class (LSTM): {predicted_class_lstm}")
plt.show()

In [ ]:
train_images_flat = train_images.reshape(len(train_images), -1)
test_images_flat = test_images.reshape(len(test_images), -1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images_flat, train_labels, test_size=0.3, random_state=42)

Model 2: Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Accuracy:", accuracy)

In [ ]:
sample_image_rf = test_images[24].reshape(1, -1)
predicted_image_rf = model.predict(sample_image_rf)[0]

plt.imshow(test_images[24], cmap='gray')
plt.title(f"Predicted Class (Random Forest): {predicted_image_rf}")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
conf = confusion_matrix(y_test, y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=conf)
display.plot()
plt.title("Confusion Matrix of Random Forest ")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred))